# Global energy statistics visualisation

This jupyter notebook has been created to visualise the global energy statistics dataset from this kaggle dataset: https://www.kaggle.com/datasets/akhiljethwa/global-electricity-statistics/

## Module Loading and Data Import

In [1]:
# For Data Analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For Data Visualization
import plotly.express as px
import plotly.io as pio

# Addidtional Imports
from termcolor import colored
import country_converter as coco
import datetime
import calendar
import json

In [2]:
# Set Plotly Template
pio.templates.default = "plotly_dark"

In [3]:
data = pd.read_csv("./data/Global Electricity Statistics.csv")
#map_json = json.load(open("./data/countries.json", "r"))

## Preparing dataframes

For our visualization we will need to preprocess our data a little.

**Get list of countries from geojson and prepare dataframe with Country and Year columns**

To get countries:
- iterate over "features" in map gejson and extract "geounit" name form feature's "properties"
- use set() to remove possible duplicates
- you can use asterisk (*) to unpack iterables into a list
- sort countries by name 

In [4]:
# countries = []
# 
# for feature in map_json["features"]:
#       countries.append(feature["properties"]["geounit"])
# 
# countries=[*set(countries)]
# countries.sort()

Now lets create dataframe, that will have columns Country and Year and **one row for every country-year combination in years 1942-2022** (those present in the dataset).
We can get this by:
- creating dataframe with countries (just turn the list into df)
- creating dataframe with years (creat list of years and turn it to df)
- creating a Caretsian product of the two dataframes

In [5]:
# country_df = pd.DataFrame(countries, columns =['Country'])
# year_df = pd.DataFrame(list(range(1980, 2022)), columns =['Year'])
# 
# countries_df = country_df.merge(year_df, how='cross')

## Filling the Dataframes (Splitting, Aggregation, Counting, ...)

In [6]:
data

,Country,Features,Region,1980,1981,1982,1983,1984,1985,1986,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Algeria,net generation,Africa,6.683,7.65,8.824,9.615,10.537,11.569,12.214,...,53.9845,56.3134,60.39972,64.68244,66.75504,71.49546,72.10903,76.685,72.73591277,77.53072719
1,Angola,net generation,Africa,0.905,0.906,0.995,1.028,1.028,1.028,1.088,...,6.03408,7.97606,9.21666,9.30914,10.203511,10.67604,12.83194,15.4,16.6,16.429392
2,Benin,net generation,Africa,0.005,0.005,0.005,0.005,0.005,0.005,0.005,...,0.04612,0.08848,0.22666,0.31056,0.26004,0.3115,0.19028,0.2017,0.22608,0.24109728
3,Botswana,net generation,Africa,0.443,0.502,0.489,0.434,0.445,0.456,0.538,...,0.33,0.86868,2.17628,2.79104,2.52984,2.8438,2.97076,3.0469,2.05144,2.18234816
4,Burkina Faso,net generation,Africa,0.098,0.108,0.115,0.117,0.113,0.115,0.122,...,0.86834,0.98268,1.11808,1.43986,1.5509,1.64602,1.6464,1.72552,1.647133174,1.761209666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,Trinidad and Tobago,distribution losses,Central & South America,0.244,0.21,0.152,0.326,0.36,0.407,0.337,...,0.239,0.234,0.245,0.248,0.253,0.274864,0.425807,0.424101,0.422757,0.422757
1606,Turks and Caicos Islands,distribution losses,Central & South America,0.00035,0.00035,0.00035,0.00035,0.00035,0.00035,0.00035,...,0,0,0,0,0,0.014,0.0125,0.0125,0.0125,0.01277172
1607,U.S. Virgin Islands,distribution losses,Central & South America,0.05243,0.05537,0.05607,0.05761,0.05789,0.05922,0.06055,...,0.063,0.06,0.06,0.06,0.065,0.065,0.05,0.051,0.051,0.051
1608,Uruguay,distribution losses,Central & South America,0.55,0.426,0.627,0.662,0.78,0.702,0.911,...,1.292,1.282,1.253,1.49,1.608,1.694491,1.557257,1.322331,1.129273,1.129273


In [7]:
data.dtypes

Country     object
Features    object
Region      object
1980        object
1981        object
1982        object
1983        object
1984        object
1985        object
1986        object
1987        object
1988        object
1989        object
1990        object
1991        object
1992        object
1993        object
1994        object
1995        object
1996        object
1997        object
1998        object
1999        object
2000        object
2001        object
2002        object
2003        object
2004        object
2005        object
2006        object
2007        object
2008        object
2009        object
2010        object
2011        object
2012        object
2013        object
2014        object
2015        object
2016        object
2017        object
2018        object
2019        object
2020        object
2021        object
dtype: object

In [8]:
melted_df = pd.melt(data, id_vars=['Country', 'Region', "Features"], var_name='Year', value_name='Value')

melted_df

,Country,Region,Features,Year,Value
0,Algeria,Africa,net generation,1980,6.683
1,Angola,Africa,net generation,1980,0.905
2,Benin,Africa,net generation,1980,0.005
3,Botswana,Africa,net generation,1980,0.443
4,Burkina Faso,Africa,net generation,1980,0.098
...,...,...,...,...,...
67615,Trinidad and Tobago,Central & South America,distribution losses,2021,0.422757
67616,Turks and Caicos Islands,Central & South America,distribution losses,2021,0.01277172
67617,U.S. Virgin Islands,Central & South America,distribution losses,2021,0.051
67618,Uruguay,Central & South America,distribution losses,2021,1.129273


In [9]:
# Step 4: Rename the columns
melted_df.columns = ['Country', 'Region', 'Features', 'Year', 'Value']

# Now 'melted_df' should have the desired transposed format
melted_df

,Country,Region,Features,Year,Value
0,Algeria,Africa,net generation,1980,6.683
1,Angola,Africa,net generation,1980,0.905
2,Benin,Africa,net generation,1980,0.005
3,Botswana,Africa,net generation,1980,0.443
4,Burkina Faso,Africa,net generation,1980,0.098
...,...,...,...,...,...
67615,Trinidad and Tobago,Central & South America,distribution losses,2021,0.422757
67616,Turks and Caicos Islands,Central & South America,distribution losses,2021,0.01277172
67617,U.S. Virgin Islands,Central & South America,distribution losses,2021,0.051
67618,Uruguay,Central & South America,distribution losses,2021,1.129273


### Preprocess data types:



In [10]:
data = melted_df

In [11]:
# Convert all "--" and "ie" into NaN

data = data.replace("--", np.nan)
data = data.replace("ie", np.nan)

In [12]:
# Convert Country, Region and Features into categorical data

data["Country"] = data["Country"].apply(lambda x: x.strip()).astype("category")
data["Region"] = data["Region"].apply(lambda x: x.strip()).astype("category")
data["Features"] = data["Features"].apply(lambda x: x.strip()).astype("category")
data["Year"] = data["Year"].astype("category")
data["Value"] = data["Value"].astype("float")

In [13]:
# Pivot the dataframe to get the desired format
new_df = data.set_index(['Country', 'Region', 'Year', 'Features']).unstack('Features').reset_index()

# Flatten the multi-level column index
new_df.columns = [col[1] if col[1] else col[0] for col in new_df.columns]

# Display the resulting dataframe
new_df

,Country,Region,Year,distribution losses,exports,imports,installed capacity,net consumption,net generation,net imports
0,Afghanistan,Asia & Oceania,1980,0.065940,0.000000,0.000000,0.374000,0.876060,0.942000,0.000000
1,Afghanistan,Asia & Oceania,1981,0.069440,0.000000,0.000000,0.427000,0.922560,0.992000,0.000000
2,Afghanistan,Asia & Oceania,1982,0.066640,0.000000,0.000000,0.427000,0.885360,0.952000,0.000000
3,Afghanistan,Asia & Oceania,1983,0.070000,0.000000,0.000000,0.450000,0.930000,1.000000,0.000000
4,Afghanistan,Asia & Oceania,1984,0.071330,0.000000,0.000000,0.450000,0.947670,1.019000,0.000000
...,...,...,...,...,...,...,...,...,...,...
9655,Zimbabwe,Africa,2017,1.728908,0.351000,2.569000,2.311252,7.838552,7.349460,2.218000
9656,Zimbabwe,Africa,2018,1.747814,0.161000,1.177000,2.455607,8.455186,9.187000,1.016000
9657,Zimbabwe,Africa,2019,1.552412,0.504000,1.612000,2.461000,7.786427,8.230839,1.108000
9658,Zimbabwe,Africa,2020,1.444983,0.355000,2.337000,2.474000,8.131027,7.594010,1.982000


In [14]:
new_df["installed capacity"] /= 1000

In [15]:
country_names_unique = new_df["Country"].unique()

In [16]:
new_df.dtypes

Country                category
Region                 category
Year                   category
distribution losses     float64
exports                 float64
imports                 float64
installed capacity      float64
net consumption         float64
net generation          float64
net imports             float64
dtype: object

In [17]:
data.dtypes

Country     category
Region      category
Features    category
Year        category
Value        float64
dtype: object

In [18]:
# Rename countries 

data["Country"] = data["Country"].replace("Former U.S.S.R.", "USSR")

In [19]:
data.dtypes

Country     category
Region      category
Features    category
Year        category
Value        float64
dtype: object

In [20]:
data["Region"].unique()

['Africa', 'Eurasia', 'Europe', 'Asia & Oceania', 'Middle East', 'North America', 'Central & South America']
Categories (7, object): ['Africa', 'Asia & Oceania', 'Central & South America', 'Eurasia', 'Europe', 'Middle East', 'North America']

### Splitting:

In [21]:
data["Features"].unique()

['net generation', 'net consumption', 'imports', 'exports', 'net imports', 'installed capacity', 'distribution losses']
Categories (7, object): ['distribution losses', 'exports', 'imports', 'installed capacity', 'net consumption', 'net generation', 'net imports']

In [22]:
data.sort_values(by=['Country'], inplace=True)

In [23]:
data_net_generation = data[data["Features"] == "net generation"].copy()
data_net_comsumption = data[data["Features"] == "net consumption"].copy()
data_imports = data[data["Features"] == "imports"].copy()
data_exports = data[data["Features"] == "exports"].copy()
data_net_imports = data[data["Features"] == "net imports"].copy()
data_installed_capacity = data[data["Features"] == "installed capacity"].copy()
data_distribution_losses = data[data["Features"] == "distribution losses"].copy()

In [24]:
data_net_generation.drop(columns=["Features"], inplace=True)
data_net_comsumption.drop(columns=["Features"], inplace=True)
data_imports.drop(columns=["Features"], inplace=True)
data_exports.drop(columns=["Features"], inplace=True)
data_net_imports.drop(columns=["Features"], inplace=True)
data_installed_capacity.drop(columns=["Features"], inplace=True)
data_distribution_losses.drop(columns=["Features"], inplace=True)

data_net_generation.reset_index(drop=True, inplace=True)
data_net_comsumption.reset_index(drop=True, inplace=True)
data_imports.reset_index(drop=True, inplace=True)
data_exports.reset_index(drop=True, inplace=True)
data_net_imports.reset_index(drop=True, inplace=True)
data_installed_capacity.reset_index(drop=True, inplace=True)
data_distribution_losses.reset_index(drop=True, inplace=True)

In [25]:
data_distribution_losses

,Country,Region,Year,Value
0,Afghanistan,Asia & Oceania,2000,0.12500
1,Afghanistan,Asia & Oceania,2018,0.64000
2,Afghanistan,Asia & Oceania,1999,0.03185
3,Afghanistan,Asia & Oceania,1998,0.03220
4,Afghanistan,Asia & Oceania,2003,0.15000
...,...,...,...,...
9655,Zimbabwe,Africa,2004,1.47300
9656,Zimbabwe,Africa,2008,0.96700
9657,Zimbabwe,Africa,2007,1.21900
9658,Zimbabwe,Africa,1987,0.50800


In [26]:
data_net_generation[data_net_generation["Country"] == "Belarus"]

,Country,Region,Year,Value
672,Belarus,Eurasia,2006,29.846080
673,Belarus,Eurasia,1983,NaN
674,Belarus,Eurasia,2004,29.339440
675,Belarus,Eurasia,1994,29.514000
676,Belarus,Eurasia,1984,NaN
677,Belarus,Eurasia,2005,29.107300
678,Belarus,Eurasia,1998,21.991000
679,Belarus,Eurasia,1986,NaN
680,Belarus,Eurasia,2000,24.536560
681,Belarus,Eurasia,2002,24.869380


There are some problems with renaming, specifically:

- Former Serbia and Montenegro not found in regex
- USSR not found in regex
- Former Yugoslavia not found in regex
- Germany, East not found in regex
- Germany, West not found in regex
- Hawaiian Trade Zone not found in regex
- Netherlands Antilles not found in regex
- U.S. Pacific Islands not found in regex
- U.S. Territories not found in regex
- Wake Island not found in regex

In [27]:
countries_short = coco.convert(names = data_net_generation['Country'], to='name_short')
country_iso3 = coco.convert(names = data_net_generation['Country'], to = "ISO3")

Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found in regex
Former Serbia and Montenegro not found i

# TODO : Add data for 

### Basic plots:

In [49]:
def get_map_prototype(year: int, feature_type: str):
    data_feature: pd.DataFrame = data[data["Features"] == feature_type].copy()
    data_feature.drop(columns=["Features"], inplace=True)
    data_feature.reset_index(drop=True, inplace=True)

    data_feature['Country_Short'] = country_iso3

    fig = px.choropleth(data_frame=data_feature,
                        locations = "Country_Short",
                        color = "Value",
                        range_color=(min(data_feature['Value']), max(data_feature["Value"])),
                        color_continuous_scale = [[0, '#0d0887'],
                        [0.01, '#46039f'],
                        [0.03, '#7201a8'],
                        [0.5, '#9c179e'],
                        [0.7, '#bd3786'],
                        [0.9, '#d8576b'],
                        [1, '#ed7953']
                        ],
                        #  focus='south america',
                        title = f'Global Electricity {feature_type} in {year}',
                        hover_name = "Country",
                        hover_data = ["Value"],)
    fig.update_layout(height=500, width=800)
    return fig

get_map_prototype(2021, "net imports")

In [50]:
def get_barplot_for_country_year_prototype(country: str):
    timeline = new_df[new_df["Country"] == country].copy()
    bars = px.bar(timeline, x="Year", y=['net generation', 'net consumption', 'imports', 'exports', 'net imports', 'installed capacity', 'distribution losses'], 
              color_discrete_sequence=["white", "red", "blue", "green", "orange", "pink", "brown"], title=f"{country}'s overall statistics",
              barmode='group', hover_data={'Year': False})
    bars.update_layout(
        xaxis_title='Year',
        yaxis_title='Amount in Billion kWh',
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
        xaxis=dict(tickmode='linear', tick0=1980, dtick=1),
        hovermode='x unified'
    )
    return bars

get_barplot_for_country_year_prototype("Afghanistan")

In [51]:
def get_lineplot_for_country_year_prototype(country: str):
    timeline = new_df[new_df["Country"] == country].copy()
    lines = px.line(timeline, x="Year", y=['net generation', 'net consumption', 'imports', 'exports', 'net imports', 'installed capacity', 'distribution losses'],
                    color_discrete_sequence=["white", "red", "blue", "green", "orange", "pink", "brown"], title=f"{country}'s overall statistics",
                    hover_data={'Year': False})
    lines.update_traces(mode='lines+markers')
    lines.update_layout(
        xaxis_title='Year',
        yaxis_title='Amount in Billion kWh',
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
        xaxis=dict(tickmode='linear', tick0=1980, dtick=1),
        hovermode='x unified'
    )
    return lines

get_lineplot_for_country_year_prototype("Afghanistan")

In [52]:
import plotly.graph_objects as go

def get_comsumption_vs_generation_prototype(country: str):
    timeline = new_df[new_df["Country"] == country].copy()
    
    fig = go.Figure()
    
    # Calculate the difference between generation and consumption
    difference = timeline["net consumption"] - timeline["net generation"]
    timeline["difference"] = difference
    
    # Add lines for generation and consumption
    fig.add_trace(go.Scatter(x=timeline["Year"], y=timeline["net generation"],
                             mode='lines+markers', name='Net Generation',
                             hoverinfo='x+y', line=dict(color='blue'),
                             hovertemplate='%{y}'))
    
    fig.add_trace(go.Scatter(x=timeline["Year"], y=timeline["net consumption"],
                             mode='lines+markers', fill='tonexty', fillcolor='rgba(0,100,80,0.2)', name='Net Consumption',
                             hoverinfo='x+y', line=dict(color='red'),
                             hovertemplate='%{y}'))
    
    # Add area for the difference
    fig.add_trace(go.Scatter(x=timeline["Year"], y=difference,
                              mode='lines', name='Consumption - Generation Difference',
                              hoverinfo='x+y', hovertemplate='%{y}'))
    
    fig.update_layout(title=f"{country}'s Net Generation vs Net Consumption",
                      xaxis_title='Year',
                      yaxis_title='Amount in Billion kWh',
                      legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
                      xaxis=dict(tickmode='linear', tick0=1980, dtick=1),
                      hovermode='x unified')
    
    return fig

get_comsumption_vs_generation_prototype("France")

In [53]:
import plotly.graph_objects as go

def visualize_country_data_difference(country: str, data_type1: str, data_type2: str):
    timeline = new_df[new_df["Country"] == country].copy()
    data_type1 = data_type1.lower()
    data_type2 = data_type2.lower()
    
    # Capitalize the labels
    data_type1_label = data_type1.capitalize()
    data_type2_label = data_type2.capitalize()
    
    # Calculate the difference between data_type1 and data_type2
    difference = timeline[data_type2] - timeline[data_type1]
    timeline["difference"] = difference
    
    fig = go.Figure()
    
    # Add lines for data_type1 and data_type2
    fig.add_trace(go.Scatter(x=timeline["Year"], y=timeline[data_type1],
                             mode='lines+markers', name=data_type1_label,
                             hoverinfo='x+y', line=dict(color='blue'),
                             hovertemplate='%{y}'))
    
    fig.add_trace(go.Scatter(x=timeline["Year"], y=timeline[data_type2],
                             mode='lines+markers', fill='tonexty', fillcolor='rgba(0,100,80,0.2)',
                             name=data_type2_label, hoverinfo='x+y', line=dict(color='red'),
                             hovertemplate='%{y}'))
    
    # Add area for the difference
    fig.add_trace(go.Scatter(x=timeline["Year"], y=difference,
                             mode='lines', name=f'{data_type2_label} - {data_type1_label} Difference',
                             hoverinfo='x+y', hovertemplate='%{y}'))
    
    fig.update_layout(title=f"{country}'s {data_type1_label} vs {data_type2_label}",
                      xaxis_title='Year',
                      yaxis_title='Amount in Billion kWh',
                      legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
                      xaxis=dict(tickmode='linear', tick0=1980, dtick=1),
                      hovermode='x unified')
    
    return fig

visualize_country_data_difference("Afghanistan", "distribution losses", "net consumption")

In [55]:

def get_import_vs_generated_plot(country: str):
    # Filter data for the given country
    country_data = new_df[new_df["Country"] == country].copy()

    # Create a stacked bar plot for imported vs generated electricity
    fig = px.bar(country_data, x="Year", y=['net generation', 'imports'],
                 color_discrete_sequence=["blue", "orange"],
                 title=f"{country}'s Imported vs Generated Electricity",
                 barmode='stack', hover_data={'Year': False},
                 labels={'variable': 'Electricity Source'})

    # Add line plot for electricity consumption
    fig.add_scatter(x=country_data["Year"], y=country_data["net consumption"],
                    mode='lines', name='Consumption',
                    line=dict(color='green', width=3))

    fig.update_layout(title=f"{country}'s electricity source location",
                      xaxis_title='Year',
                      yaxis_title='Amount in Billion kWh',
                      legend=dict(yanchor="top", y=0.99,
                                  xanchor="left", x=0.01),
                      xaxis=dict(tickmode='linear', tick0=1980, dtick=1),
                      hovermode='x unified')

    return fig

# Example usage:
get_import_vs_generated_plot("Afghanistan")

# Main dashly app

In [33]:
from dash import Dash, html, dcc, Input, Output, dash_table, State, callback_context, no_update

#import logging, sys
#logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

app = Dash(__name__, suppress_callback_exceptions=True)

main_map_state = True

#*******APP LAYOUT**************

app.layout = html.Div(

    style={'backgroundColor':'#323130',
        'height': '100%',
        'color': 'white',
        'margin': 0,
        'padding': '15px' 
    }, 

    children=[
        html.H1(
            children='Global Electricity Statistics',
            style={
                'textAlign': 'center',
                'color': 'white'
            }
        ),

        html.Div(children='''Dash: A web application framework for Python.''',
                style={
                    'textAlign': 'center',
                    'color': 'white'
                }
        ),

        dcc.Dropdown(
            id='country_dropdown',
            options=[{'label': i, 'value': i} for i in country_names_unique],
            style={'color': 'black'}
        ),

        html.Div([
            
            dcc.Graph(
                id='main_map',
                #figure=fig,
                style={'width': '100%', 'display': 'inline-block'}
            ),
        
        ], id="main_component"),
        
        html.Div(id="controls", children= [    
            html.Div([
                html.Div([
                    dcc.Markdown('**Features**'),
                    dcc.Dropdown(options=[{'label': feat, 'value': feat} for feat in data["Features"]],
                                value='net generation',
                                id='feature_type',
                                style={'color': 'black'}),
                ], style={'width': '20%', 'display': 'inline-block', 'margin-right': '2%'}),

                html.Div([
                    dcc.Markdown('**Years**'),
                    dcc.Slider(1980, 2021, step=1, value=2021, id='slider',
                            marks={i: '{}'.format(i) for i in range(1980, 2021, 10)},
                            tooltip={'placement': 'bottom', 'always_visible': True}),
                ], style={'width': '55%', 'display': 'inline-block', 'margin-right': '2%'}),

                html.Div([
                    html.Button('Compare countries', id='comparison_button', style={'margin-right': '5%', 'height': '100%'}),
                    html.Button('Main map mode', id='main_map_button', style={'height': '100%'}),
                ], style={'width': '20%', 'display': 'inline-block'}),
            ], style={'height': '100vh'}),
        ]),
    ],
)

#**************FUNCTIONS*****************************

def get_map(year: int, feature_type: str, country: str = None):
    if year is None:
        year = 2021
        
    if feature_type is None:
        feature_type = "net generation"
    
    data_feature: pd.DataFrame = data[data["Features"] == feature_type].copy()
    data_feature.drop(columns=["Features"], inplace=True)
    data_feature.reset_index(drop=True, inplace=True)

    data_feature['Country_Short'] = country_iso3

    fig = px.choropleth(data_frame=data_feature,
                        locations = "Country_Short",
                        color = "Value",
                        range_color=(min(data_feature['Value']), max(data_feature["Value"])),
                        color_continuous_scale = [[0, '#0d0887'],
                        [0.01, '#46039f'],
                        [0.03, '#7201a8'],
                        [0.5, '#9c179e'],
                        [0.7, '#bd3786'],
                        [0.9, '#d8576b'],
                        [1, '#ed7953']
                        ],
                        #  focus='south america',
                        title = f'Global Electricity {feature_type} in {year}',
                        hover_name = "Country",
                        hover_data = ["Value"],)
    fig.update_layout(height=500, width=800)
    
    # TODO: Set the geographical scope to the clicked country
    # if country is not None:
    #     fig.update_geos(scope=country.lower())
    
    return fig

def get_barplot_for_country_year(country: str):
    if country is None:
        country = "Belgium"
    
    timeline = new_df[new_df["Country"] == country].copy()
    bars = px.bar(timeline, x="Year", y=['net generation', 'net consumption', 'imports', 'exports', 'net imports', 'installed capacity', 'distribution losses'], 
              color_discrete_sequence=["white", "red", "blue", "green", "orange", "pink", "brown"], title=f"{country}'s statistics over the years",
              barmode='group')
    bars.update_layout(
        xaxis = dict(
            tickmode = 'linear',
            tick0 = 1980,
            dtick = 1
        ), 
        height=500, 
        width=800
    )
    return bars

#*************CALLBACKS*****************************************

#radio/slider->map
@app.callback(
    Output('main_map', 'figure'),
    Input('slider', 'value'),
    Input('feature_type', 'value')
)
def update_map(year, feature_type):
    print("update_map")
    fig = get_map(year, feature_type)
    return fig


@app.callback(
    Output('country_features_over_time', 'figure'),
    Input('country_dropdown', 'value')
)
def update_country_features_over_time(country):
    print("update_country_features_over_time")
    fig = get_barplot_for_country_year(country)
    return fig


@app.callback(
    Output('country_dropdown', 'value'),
    Input('main_map', 'clickData'),
    prevent_initial_call=True
)
def update_dropdown_value(clickData):
    print("update_dropdown_value")
    if clickData is not None:
        tmp = clickData['points'][0]['hovertext']
        clickData = None
        return tmp
    else:
        # Return default value if no click data
        return no_update
        
    
@app.callback(Output('main_component', 'children', allow_duplicate=True),
              Input('main_map', 'clickData'),
              State('slider', 'value'),
              State('feature_type', 'value'),
              Input('country_dropdown', 'value'),
              prevent_initial_call=True)
def country_specific_mode(clickData, selected_year, selected_feature, country_dropdown_value):
    print("country_specific_mode")
    
    print(clickData)
    
    global main_map_state
    if not main_map_state:
        main_map_state = True
        return no_update
    
    country = 'Belgium'
    if country_dropdown_value is not None:
        country = country_dropdown_value    
    if clickData is not None and country_dropdown_value is None:
        country = clickData['points'][0]['hovertext']
        clickData = None
    fig = get_barplot_for_country_year(country)
    
    map = get_map(selected_year, selected_feature, country)
    return [dcc.Graph(figure=fig, id='country_features_over_time'), dcc.Graph(figure=map, id='main_map')]


@app.callback(
    Output('main_component', 'children', allow_duplicate=True),
    Input('comparison_button', 'n_clicks'),
    Input('main_map_button', 'n_clicks'),
    Input('country_dropdown', 'value'),
    State('slider', 'value'),
    State('feature_type', 'value'),
    prevent_initial_call=True
)
def update_main_component(compare_clicks, main_map_clicks, selected_country, selected_year, selected_feature):
    print("update_main_component")
    ctx = callback_context

    if not ctx.triggered:
        return no_update
    
    print("Context:", ctx.triggered_prop_ids)

    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    print("Switch:", end=" ")
    if button_id == 'comparison_button':
        # Comparison mode
        print("Comparison mode")
        fig = get_barplot_for_country_year(selected_country)
        return dcc.Graph(figure=fig, id='country_features_over_time')
    elif button_id == 'main_map_button':
        # Main map mode
        print("Main map mode")
        global main_map_state
        main_map_state = False
        fig = get_map(selected_year, selected_feature)
        return dcc.Graph(figure=fig, id='main_map')
    else:
        # Country-specific mode
        print("Country-specific mode")
        fig = get_barplot_for_country_year(selected_country)
        return dcc.Graph(figure=fig, id='country_features_over_time')



#********RUNNING THE APP*************************************************
if __name__ == '__main__':
    app.run_server(debug=True, 
                   port="8999", 
                   jupyter_mode="external", # inline/tab/external jupyter_mode="external"
    )

Dash app running on http://127.0.0.1:8999/


TODO:

- OK Add country detail screen on country click 
- Add more graphs/components for the whole visualisation - mainly the country detail screen and comparison screen
- Do the add button to have multiple countries for comparison
- Fixup the styling
- NOK Add zooming to the map - can't be done with plotly express